In [1]:
from clease.calculator import attach_calculator

In [2]:
from clease import Concentration, CEBulk, NewStructures

In [3]:
from clease.montecarlo import Montecarlo

In [4]:
from clease.tools import update_db

In [5]:
from ase.calculators.emt import EMT

In [6]:
from ase.db import connect

# Select concentration

In [7]:
conc = Concentration(basis_elements=[['Au', 'Cu']])
conc.set_conc_ranges(ranges=[[(0, 0.5), (0.5, 1)]])

# Setup cluster expansion

In [8]:
setting = CEBulk(
    crystalstructure='fcc',
    a=3.8,
    supercell_factor=27,
    concentration=conc,
    db_name="aucu_bulk.db",
    max_cluster_size=4,
    max_cluster_dia=[6.0, 5.0, 5.0],
    basis_function='polynomial'
)

Generating cluster data for template with size: 1,0,0x0,1,0x0,0,1. (1 of 9)
Generating cluster data for template with size: 1,0,0x0,1,0x0,0,2. (2 of 9)
Generating cluster data for template with size: 2,0,0x0,2,0x0,0,2. (3 of 9)
Generating cluster data for template with size: 2,0,0x0,2,0x0,0,3. (4 of 9)
Generating cluster data for template with size: 2,0,0x0,2,0x0,0,4. (5 of 9)
Generating cluster data for template with size: 2,0,0x0,2,0x0,0,5. (6 of 9)
Generating cluster data for template with size: 2,0,0x0,3,0x0,0,3. (7 of 9)
Generating cluster data for template with size: 2,0,0x0,3,0x0,0,4. (8 of 9)
Generating cluster data for template with size: 3,0,0x0,3,0x0,0,3. (9 of 9)


# Generate structures 

In [9]:
ns = NewStructures(
    setting=setting, 
    generation_number=0, 
    struct_per_gen=10
)

In [10]:
ns.generate_initial_pool()

Generating one structure per concentration where the number of an element is at max/min
Generating 9 random structures (struct_per_gen=10, 1 present)
Generated 1 random structures
Generated 2 random structures
Generated 3 random structures
Generated 4 random structures
Generated 5 random structures
Generated 6 random structures
Generated 7 random structures
Generated 8 random structures
Generated 9 random structures


In [11]:
atoms = connect('aucu_bulk.db').get(id=11).toatoms()

In [12]:
ns.generate_random_structures(atoms)

There are 10 structures in generation 0 in DB and struct_per_gen = 10. No more structures generated.


# Calculate energies using EMT

In [13]:
calc = EMT()

In [14]:
db_name = "aucu_bulk.db"
db = connect(db_name)

In [15]:
for row in db.select(converged=False):
    atoms = row.toatoms()
    atoms.set_calculator(calc)
    atoms.get_potential_energy()
    update_db(uid_initial=row.id, final_struct=atoms, db_name=db_name)

# Training / Fitting 

In [16]:
setting = CEBulk(
    crystalstructure='fcc',
    a=3.8,
    supercell_factor=27,
    concentration=conc,
    db_name="aucu_bulk.db",
    max_cluster_size=4,
    max_cluster_dia=[6.0, 5.0, 5.0],
    basis_function='polynomial'
)

# Monte Carlo

In [17]:
eci = {'c0': -1.0, 'c1_0': 0.1, 'c2_d0000_0_00': -0.2}

In [18]:
atoms = setting.atoms.copy()*(5, 5, 5)
atoms = attach_calculator(setting, atoms=atoms, eci=eci)

Template that matches the size of passed atoms not found. Generating...


In [19]:
atoms[0].symbol = 'Cu'
atoms[1].symbol = 'Cu'
atoms[2].symbol = 'Cu'

In [20]:
T = 500
mc = Montecarlo(atoms, T)
mc.run(steps=1000)

Probing energy bias using 250 MC steps...
Energy after probing: -136.1
Bias subtracted from empty cluster...
Reached maximum number of steps (1000 mc steps)
Empty cluster ECI reset to original value...


In [21]:
thermo = mc.get_thermodynamic_quantities()

In [22]:
thermo

{'energy': -136.07133333333334,
 'heat_capacity': 0.0002663573054610469,
 'energy_var': 0.005738222222222279,
 'temperature': 500,
 'Cu_conc': 0.024,
 'Au_conc': 0.976,
 'timestamp': '2019-10-17 14:27:32',
 'python_version': '3.7.3'}